In [1]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpMinimize, LpVariable, lpSum

In [2]:
df = pd.read_csv("df_for_pulp_testing.csv", index_col=0)

In [67]:
from pulp import LpProblem, LpVariable, LpMinimize
import pandas as pd

# Define the problem
prob = LpProblem("Minimize_Cost", LpMinimize)
decision_variables = []
nc = []
c_num = 1

# Index of the first food column
first_food_idx = df.columns.get_loc("2% Milk")

# Define decision variables for each food item
for food in df.columns[first_food_idx:]:
    decision_variables.append(LpVariable(f"{food}", lowBound=0))

# Objective function: minimize the sum of decision variables (or add specific costs if needed)
prob += sum(decision_variables)

# Add constraints based on nutrient requirements
for i, row in df.iterrows():
    constraint = 0
    for j, food in enumerate(df.columns[first_food_idx:]):
        if pd.notna(row[food]):  # Ensure row[food] is not NaN
            constraint += decision_variables[j] * row[food]

    # Check if target, ll, or ul are non-NaN and add the corresponding constraint
    if pd.notna(row.get("target")):
        nc.append((row.nutrient_name, "target", c_num))
        prob += constraint == row.target
        c_num += 1
    elif pd.notna(row.get("ll")):
        nc.append((row.nutrient_name, "ll", c_num))
        prob += constraint >= row.ll
        c_num += 1
    elif pd.notna(row.get("ul")):
        nc.append((row.nutrient_name, "ul", c_num))
        prob += constraint <= row.ul
        c_num += 1

In [71]:
shadow_prices = []
prob.solve()
for v in prob.variables():
    if v.varValue != 0:
        shadow_prices.append([v.name, v.varValue])
pd.DataFrame(shadow_prices, columns=["food", "amount"])

,food,amount
0,2%_Milk,0.196551
1,Beef_Liver,0.300712
2,Broccoli,0.287971
3,Carrots,0.072285
4,Peanuts,0.205089
5,Pink_Salmon,0.380271
6,Skim_Milk,0.439785
7,Split_Peas,0.515895
8,Sunflower_seeds,0.140602
9,Watermelon,0.291859


In [70]:
pis = []

for name, constraint in prob.constraints.items():
    pis.append(constraint.pi)
constraint_shadow_prices = pd.DataFrame(
    nc, columns=["nutrient_name", "constraint_type", "constraint_num"]
).assign(shadow_price=pis)
constraint_shadow_prices.sort_values(by="shadow_price", ascending=False)

,nutrient_name,constraint_type,constraint_num,shadow_price
33,EPA+DHA,ll,34,0.151358
20,Vitamin D (D2 + D3),ll,21,0.038855
0,Protein,ll,1,0.015843
34,Omega-6,ll,35,0.011354
22,"Vitamin C, total ascorbic acid",ll,23,0.003833
1,Total lipid (fat),ll,2,0.002926
19,Vitamin E (alpha-tocopherol),ll,20,0.001942
18,"Carotene, alpha",ll,19,0.000042
21,Lycopene,ll,22,0.000030
2,"Carbohydrate, by difference",ll,3,0.000000
